In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os, sys
sys.path.insert(0,"./python")
from transaction_simulator import *

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# 1.) Load parameters

In [ ]:
import sys
from datawand.parametrization import ParamHelper

In [ ]:
ph = ParamHelper('..', 'LNGraph', sys.argv)

experiment_id = ph.get("experiment_id")
snapshot_id = ph.get("snapshot_id")
amount_sat = 200000#15USD #ph.get("amount_sat")
num_transactions = 2000#ph.get("num_transactions")

In [ ]:
experiment_id = ph.get("experiment_id")
snapshot_id = ph.get("snapshot_id")
amount_sat = ph.get("amount_sat")
num_transactions = ph.get("num_transactions")
eps = ph.get("eps")
day_interval = ph.get("day_interval")
drop_disabled = ph.get("drop_disabled")
drop_low_cap = ph.get("drop_low_cap")
with_depletion = ph.get("with_depletion")

In [ ]:
data_dir = ph.get("data_dir")
output_dir = "%s/simulations/%s/%s" % (data_dir, snapshot_id, experiment_id)
print(output_dir)

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 2.) Load data

In [ ]:
snapshots = pd.read_csv("%s/directed_graphs/directed_temporal_multi_edges_%idays.csv" % (data_dir, day_interval))

In [ ]:
snapshots["snapshot_id"].min(), snapshots["snapshot_id"].max()

In [ ]:
snapshots.head(2)

In [ ]:
node_meta = pd.read_csv("%s/node_meta_with_labels.csv" % data_dir)

In [ ]:
providers = list(node_meta["pub_key"])
len(providers)

In [ ]:
snapshots.head()

In [ ]:
edges = snapshots[snapshots["snapshot_id"]==snapshot_id]

In [ ]:
edges.groupby("src")["capacity"].sum().mean()

In [ ]:
edges.groupby("src")["capacity"].sum().median()

In [ ]:
edges.groupby("src")["capacity"].sum().hist(range=(0,10**8), bins=50)

# 3.) Simulation

In [ ]:
simulator = TransactionSimulator(edges, providers, amount_sat, num_transactions, drop_disabled=drop_disabled, drop_low_cap=drop_low_cap, eps=eps, with_depletion=with_depletion)

In [ ]:
transactions = simulator.transactions

In [ ]:
print(len(transactions["source"].unique()), len(transactions["target"].unique()))

In [ ]:
transactions["source"].value_counts().hist(bins=50)

In [ ]:
transactions["target"].value_counts().hist(bins=50)

- 10K trans, with_depletion=True: 3min 17s

In [ ]:
%%time
shortest_paths, alternative_paths, all_router_fees, _ = simulator.simulate(weight="total_fee", with_node_removals=True, max_threads=16)

In [ ]:
total_income, total_fee = simulator.export(output_dir)

# 4. Some stats

total_income["fee"].sum()

num_router_nodes = len(all_router_fees["node"].unique())

activity_cnt = all_router_fees["node"].value_counts()
active_routers = list(activity_cnt[activity_cnt > 10].index)

len(active_routers) / num_router_nodes

fee_subset = all_router_fees[all_router_fees["node"].isin(active_routers)]

activity_df = pd.DataFrame()
activity_df["traffic"] = fee_subset["node"].value_counts()
activity_df["mean_fee"] = fee_subset.groupby("node")["fee"].mean()

### Router nodes with high traffic do NOT ask higher fees!!!

- there is higher negative correlation if drop_disabled=True

activity_df.corr(method="spearman")

# 5. Analyze optimal routing fee for nodes

total_income.set_index(total_income["node"]).head()

## a.) optimal routing fee for all nodes

In [ ]:
opt_fees_df, p_altered = calc_optimal_base_fee(shortest_paths, alternative_paths, all_router_fees)

In [ ]:
opt_fees_df["income_diff"].mean()

In [ ]:
opt_fees_df.set_index("node").sort_values("income_diff", ascending=False).head(10)

In [ ]:
opt_fees_df.to_csv("%s/opt_fees.csv" % output_dir, index=False)

## b.) Visualization of optimal routing fee calculation

#### top 5 routers

In [ ]:
for n in list(p_altered["node"].value_counts()[:5].index):
    _, _, _, _, _ = calculate_max_income(n, p_altered, shortest_paths, all_router_fees, visualize=True)

#### rnd 5 routers

In [ ]:
for n in list(p_altered.sample(5)["node"]):
    _, _, _, _, _ = calculate_max_income(n, p_altered, shortest_paths, all_router_fees, visualize=True)